In [3]:
import urllib2
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LassoCV
from math import sqrt
import matplotlib.pyplot as plt

In [4]:
target_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = urllib2.urlopen(target_url)

xList = []
labels = []
names = []
firstline = True
for line in data:
    if firstline:
        names = line.strip().split(";")
        firstline = False
    else:
        row = line.strip().split(";")
        labels.append(float(row[-1]))
        row.pop()
        floatRow = [float(num) for num in row]
        xList.append(floatRow)
        
# Normalize columns in x and labels

nrows = len(xList)
ncols = len(xList[0])

# Calculate means and variance
xMeans = []
xSD = []
for i in range(ncols):
    col = [xList[j][i] for j in range(nrows)]
    mean = sum(col)/nrows
    xMeans.append(mean)
    colDiff = [(xList[j][i] - mean) for j in range(nrows)]
    sumSq = sum([colDiff[i] * colDiff[i] for i in range(nrows)])
    stdDev = sqrt(sumSq/nrows)
    xSD.append(stdDev)
    
# Use mean and SD to normalize xList
xNormalized = []
for i in range(nrows):
    rowNormalized = [(xList[i][j] - xMeans[j])/xSD[j] for j in range(ncols)]
    xNormalized.append(rowNormalized)
    
# Normalize labels
meanLabel = sum(labels)/nrows
sdLabel = sqrt(sum([(labels[i] - meanLabel) * (labels[i] - meanLabel) for i in range(nrows)])/nrows)
labelNormalized = [(labels[i] - meanLabel)/sdLabel for i in range(nrows)]

#Convert into Numpy for input to sklearn packages

# Unormalzied Labels
#Y = np.array(labels)

# Unnormalized X's
#X = np.array(xList)

# Normalized labels
Y = np.array(labelNormalized)

# Normalized X's
X = np.array(xNormalized)

alphas, coefs, _ = linear_model.lasso_path(X,Y, return_models=False)

print (alphas)
print (coefs)

plt.plot(alphas,coefs.T)
plt.ylabel("Coefficients")
plt.axis("tight")
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.show()

URLError: <urlopen error [Errno 60] Operation timed out>

In [ ]:
nattr,nalpha = coefs.shape

# Find ceofficient ordering
nzList = []
for iAlpha in range(1, nalpha):
    coefList = list(coefs[:, iAlpha])
    nzCoef = [index for index in range(nattr) if coefList[index] != 0.0]
    for q in nzCoef:
        if not(q in nzList):
            nzList.append(q)
            
nameList = [names[nzList[i]] for i in range(len(nzList))]
print ("Attrinutes ordered by how early they entered the model", nameList)

In [ ]:
# Find coeff corrosponding to the best alpha value 
# Corrosponding to Normalized X and Y best alpha value is 0.013561387700964642

alphaStar = 0.013561387700964642
indexLTalphaStar = [index for index in range(100)if alphas[index] > alphaStar]
indexStar = max(indexLTalphaStar)

# Here the set of coefficients to deploy 
coefStar = list(coefs[:, indexStar])
print ("Best coefficient Values", coefStar)

# The coefficients on Normalized attributes gives another slightly different ordering

absCoef = [abs(a) for a in coefStar]

# sort by magnitude
coefSorted = sorted(absCoef, reverse=True)

idxCoefSize = [absCoef.index(a) for a in coefSorted if not(a == 0.0)]

nameList2 = [names[idxCoefSize[i]] for i in range(len(idxCoefSize))]

print ("Attributes ordered by Coef Size at Optimum Alpha", nameList2)